In [2]:
import pandas as pd
from pathlib import Path




In [ ]:
folder = Path('source/')
files = [f.name for f in folder.iterdir() if f.is_file()]

['savedrecs (1).xls', 'savedrecs (10).xls', 'savedrecs (100).xls', 'savedrecs (11).xls', 'savedrecs (12).xls', 'savedrecs (13).xls', 'savedrecs (14).xls', 'savedrecs (15).xls', 'savedrecs (16).xls', 'savedrecs (17).xls', 'savedrecs (18).xls', 'savedrecs (19).xls', 'savedrecs (2).xls', 'savedrecs (20).xls', 'savedrecs (21).xls', 'savedrecs (22).xls', 'savedrecs (23).xls', 'savedrecs (24).xls', 'savedrecs (25).xls', 'savedrecs (26).xls', 'savedrecs (27).xls', 'savedrecs (28).xls', 'savedrecs (29).xls', 'savedrecs (3).xls', 'savedrecs (30).xls', 'savedrecs (31).xls', 'savedrecs (32).xls', 'savedrecs (33).xls', 'savedrecs (34).xls', 'savedrecs (35).xls', 'savedrecs (36).xls', 'savedrecs (37).xls', 'savedrecs (38).xls', 'savedrecs (39).xls', 'savedrecs (4).xls', 'savedrecs (40).xls', 'savedrecs (41).xls', 'savedrecs (42).xls', 'savedrecs (43).xls', 'savedrecs (44).xls', 'savedrecs (45).xls', 'savedrecs (46).xls', 'savedrecs (47).xls', 'savedrecs (48).xls', 'savedrecs (49).xls', 'savedrecs (

In [8]:
d = pd.read_excel('source/' + files[0])
d.columns

Index(['Publication Type', 'Authors', 'Book Authors', 'Book Editors',
       'Book Group Authors', 'Author Full Names', 'Book Author Full Names',
       'Group Authors', 'Article Title', 'Source Title', 'Book Series Title',
       'Book Series Subtitle', 'Language', 'Document Type', 'Conference Title',
       'Conference Date', 'Conference Location', 'Conference Sponsor',
       'Conference Host', 'Author Keywords', 'Keywords Plus', 'Abstract',
       'Addresses', 'Affiliations', 'Reprint Addresses', 'Email Addresses',
       'Researcher Ids', 'ORCIDs', 'Funding Orgs', 'Funding Name Preferred',
       'Funding Text', 'Cited References', 'Cited Reference Count',
       'Times Cited, WoS Core', 'Times Cited, All Databases',
       '180 Day Usage Count', 'Since 2013 Usage Count', 'Publisher',
       'Publisher City', 'Publisher Address', 'ISSN', 'eISSN', 'ISBN',
       'Journal Abbreviation', 'Journal ISO Abbreviation', 'Publication Date',
       'Publication Year', 'Volume', 'Issue', 'Pa